# Install packages
Please install all any needed packages here.

In [ ]:
%pip install numpy matplotlib pandas
%pip install torch
%pip install torch-geometric
%pip install torch-scatter 
%pip install scikit-learn

# Import packages

Please import all the needed packages here

In [ ]:
import torch
import numpy as np
import os
from os.path import join
import pandas as pd
from torch_geometric.data import Dataset
from torch_geometric.data import Data
from os import listdir
import json
from os.path import isfile
from random import sample
import random
import torch.nn.functional as F

In [ ]:
torch.manual_seed(10)

import warnings
warnings.filterwarnings("ignore")

# Utility functions

All the utility functions required for data processing, model training and post processing are defined below

In [ ]:
def min_max_scaler(x, min_val, max_val, device):
    
    """Scales the data using the max and min values.
    
    Parameters
    -----------
    x : an N dimensional tensor, can be node attributes or target. 
    min_val : column wise min of x, array
    max_val : column wise max of x, array
    device : GPU or CPU device
    
    Ouput
    ------
    a tensor of the same size as x with scaled values
    """ 
    return (x.to(device)-min_val.to(device))/(max_val.to(device)-min_val.to(device))
    
def min_max_descaler(x, min_val, max_val, device):
    
    """Descale the data using the max and min values.
    
    Parameters
    -----------
    x : an N dimensional tensor, can be node attributes or target. 
    min_val : column wise min of x, array
    max_val : column wise max of x, array
    device : GPU or CPU device
    
    Ouput
    ------
    a tensor of the same size as x with descaled values
       
    """ 
    return (x.to(device)*(max_val.to(device)-min_val.to(device)))+ min_val.to(device)

def get_max_min(data):
    
    """Generating the maximum and minimum values of the training data.
    
    This is needed if you need to scale the data
    
    Parameters
    -----------
    data : contains graph data created using Data functiom
    
    Ouput
    ------
    X_max_c, Y_max_c, E_max_c : maximum of X, Y, and  E (node attributes, target and edge attributes) for the graph, float values
    X_min_c, Y_min_c, E_min_c : minimum of X, Y, and  E (node attributes, target and edge attributes) for the graph, float values    
    """ 
       
    train_loader = DataLoader(data, batch_size=20)
    
    X_c=[]
    Y_c=[]
    E_c=[]
    
    
    for data in train_loader:
        X_c.append(data.x.detach().cpu().numpy())
        Y_c.append(data.y.detach().cpu().numpy())
        E_c.append(data.edge_attr.detach().cpu().numpy())
        
        
    # graph max and min values
    X_c = np.vstack(X_c)
    Y_c = np.vstack(Y_c)
    E_c = np.vstack(E_c)
    X_max_c = torch.Tensor(np.max(X_c,0))
    Y_max_c = torch.Tensor(np.max(Y_c,0))
    E_max_c = torch.Tensor(np.max(E_c,0))
    X_min_c = torch.Tensor(np.min(X_c,0))
    Y_min_c = torch.Tensor(np.min(Y_c,0))
    E_min_c = torch.Tensor(np.min(E_c,0))
    
    
    return X_max_c, Y_max_c, E_max_c, X_min_c, Y_min_c, E_min_c



# Graph data creation

This section creates the graph data using the Data function of pytorch geometric.


In [ ]:
## function for farthest point sampler
def fps(node_num,x,y,npoint): 
    N = len(node_num)
    centroids = torch.zeros(npoint, dtype=torch.long)#.to(device)
    distance = torch.ones(N) * 1e10 #. #.to(device)
    farthest = torch.randint(0, N, (1,), dtype=torch.long)[0]#.to(device)
    for i in range(npoint):
        centroids[i] = farthest
        centroid_x = x[farthest]
        centroid_y = y[farthest]
        dist = torch.from_numpy((x-centroid_x)**2 + (y-centroid_y)**2)
        dist = dist.type(torch.float)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return node_num[centroids]


## function for fake edge creation
def fake_edge_creation(num_edges, node_list, edge_list, sample_algo='random'):
    
    '''This function creates fake edges.
    num_edges is the number of fake edges needed.
    It creates 2*num_edges edges (as they are undirected)'''
    
    fake_edge_list = []
    
    if(sample_algo=='random'):
        # gets the list of node numbers
        node_num = list(range(0,node_list.shape[0]))
        while(len(fake_edge_list)<num_edges): #checking if we got the required number of fake edges
            #sample 2 sample nodes from the node_num list
            #and it checks if that pair is present in the existing list of edges
            #if its not there, add this pair to the fake_edge_list
            sampled_nodes = sample(node_num,2)
            if(~np.isin(sampled_nodes[1],edge_list[edge_list[:,0]==sampled_nodes[0],1])):
                fake_edge_list.append(sampled_nodes)
    else:
        node_num = list(range(0,node_list.shape[0]))
        x = np.array(node_list[:,0])
        y = np.array(node_list[:,1])
        sampled_nodes = fps(node_num, x,y,num_edges+1)
        for i in range(len(sampled_nodes)-1):
            if(~np.isin(sampled_nodes[i+1],edge_list[edge_list[:,0]==sampled_nodes[i],1])):
                fake_edge_list.append([sampled_nodes[i],sampled_nodes[i+1]])
    return fake_edge_list


def fake_edge_attribute_creation(edge_list,node_data):
    
    '''This function creates the attributes for the edges.
    This includes x2-x1, y2-y1, distance, inverse of the deltas.'''
    
    edge_list = pd.DataFrame(edge_list)  #contains the list of fake edges of size Nx2
    edge_list.columns = ['node_1', 'node_2']
    
    node_data = np.array(node_data).reshape(-1,2)
    node_data = np.append(node_data, list(range(0,node_data.shape[0])), 1)
    
    end_node_level_data = pd.DataFrame(node_data)
    end_node_level_data.columns = ['x','y','node'] #contains x and y coordinates and node numbers

    ## Getting the nodal coordinates for the pairs of nodes of the fake edges
    edge_list = pd.merge(edge_list, end_node_level_data[['x','y','node']], left_on='node_1', right_on='node')
    edge_list.rename(columns = {'x':'x_1', 'y':'y_1'}, inplace = True)

    edge_list = pd.merge(edge_list, end_node_level_data[['x','y','node']], left_on='node_2', right_on='node')
    edge_list.rename(columns = {'x':'x_2', 'y':'y_2'}, inplace = True)

    edge_list = edge_list.drop(columns=['node_x', 'node_y'])

    ## Creating necessary edge attributes, you can change this based on your problem!!!
    edge_list['delta_x'] = edge_list['x_2'] - edge_list['x_1']
    edge_list['delta_y'] = edge_list['y_2'] - edge_list['y_1']
    edge_list['inv_delta_x'] = 1./edge_list['delta_x']
    edge_list['inv_delta_y'] = 1./edge_list['delta_y']

    edge_list['distance'] = np.sqrt(edge_list['delta_x']**2 + edge_list['delta_y']**2)


    edge_list['fake_edge'] = [1]*edge_att.shape[0]
    
    edge_list = edge_att.drop_duplicates().reset_index()
    
    return edge_list

def create_fake_edge_data(fake_edge_num, node_list,edge_list):
    
    '''This function creates the fake edge data frame
    with the list of fake edges and their attributes'''
    edge_list = np.array(edge_list).reshape(-1,2)
    fake_edge_list = fake_edge_creation(fake_edge_num, node_list, edge_list, sample_algo='random')
    fake_edge_list=np.array(fake_edge_list).reshape(-1,2)
    reverse_fake_edge = np.column_stack((fake_edge_list[:,1], fake_edge_list[:,0]))
    fake_edge_list = np.concatenate((fake_edge_list, reverse_fake_edge), axis=0)
    fake_edge_data_df = pd.DataFrame(fake_edge_list)
    fake_edge_att = fake_edge_attribute_creation(fake_edge_data_df,node_list)
    
    return fake_edge_list, fake_edge_att


In [ ]:
def process_graph_data(source_dir, sim, fake_perc):
    
    """Loading the data from the source directory and returns the data
    
    Parameters
    -----------
    source_dir : source directory which contains the original graph data, with the name format 'run_{sim}'.
    sim : simulation/run number, to identify different simulations, int
    fake_perc : % of augmented edges
    Ouput
    ------
    data - Pytorch geometric Data object
       
    """ 
    
    data = torch.load(osp.join('{}/run_{}.pt'.format(source_dir,sim)))
    
    node_data = data.x[:,:2]
    edge_data = data.edge_index.t().reshape(-1,2)

    fake_edge_num = round(edge_data.shape[0]*fake_perc)
    fake_edge_list, fake_edge_att = create_fake_edge_data(fake_edge_num, node_data,edge_data)
    fake_edge_att = np.array(fake_edge_att)
    
    edge_attr = np.array(data.edge_attr)
    
    #adding fake_flag=0 to the edge_attributes of real edges
    edge_attr = np.append(edge_attr, np.zeros(edge_attr.shape[0]),1)
    
    # adding fake edges and the attributes to the real edge data
    edge_attr = np.concatenate([edge_attr,fake_edge_att],axis=0)
    edge_connection = np.concatenate([np.array(edge_data),fake_edge_list],axis=0)
    
    edge_attr = torch.Tensor(edge_attr)
    edge_connection = torch.Tensor(edge_connection, dtype=torch.long)
    
    data.edge_index = edge_connection.t().contiguous()
    data.edge_attr = edge_attr
    
    #DO THE REST OF THE PROCESSING BASED ON YOUR DATA!!
    
    return data

In [ ]:
class GraphDataset(Dataset):
    
    """
    Class to create custom graph dataset compatible for EAGNN
    
    Every graph data generated using this class contains graph of Data object.
    
    """ 
    
    def __init__(self, root, source_dir, sim_list, fake_perc, test=False, transform=None, pre_transform=None):
        
        self.root = root # root directory where procssed data is stored
        self.sims = sim_list # list of simulation numbers (different for train and test data)
        self.test = test # flag to identify test data
        self.source_dir = source_dir # source directory for raw data
        self.fake_perc = fake_perc
        
        super(GraphDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return []

    def download(self):
        pass
    
    def len(self):
        return len(self.sims)
    
    def process(self):
        i = 0
        
        for i in range(len(self.sims)):
            print("processing simulation {}!".format(i))
            data = process_graph_data(self.source_dir, self.sims[i],self.fake_perc)
            
            if(self.test):
                torch.save(data, osp.join(self.root, 'processed/test_run_{}.pt'.format(i)))
                
            else:
                torch.save(data, osp.join(self.root, 'processed/run_{}.pt'.format(i)))

    def get(self,idx):
        
        if(self.test):
            i = idx 
            data = torch.load(osp.join(self.root, 'processed/test_run_{}.pt'.format(i)))
            
        else:
            i = idx 
            data = torch.load(osp.join(self.root, 'processed/run_{}.pt'.format(i)))
            
        return data 

# Data


In [ ]:
# getting the list of simulation numbers from the dataset
### change this based on your data!!!
mypath = 'dataset/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
file_nums=[]
for file in onlyfiles:
    file_nums.append(int(file.split('_')[1]))
file_nums = np.unique(np.array(file_nums)) 

# defining the root and source directory
# root directory/processed is the folder where the processed graph data is stored
# source directory contains the raw graph data
root_dir = "dataset/"
source_dir = "dataset"

# 500 simulations are used for training and 300 for testing
# you can changes these numbers
train_list = file_nums[:500]
val_list = file_nums[500:]

#change batch size and percentage of fake edges, if needed
batch_size = 2
fake_perc = 0.1

In [ ]:
# Processing training and validation datasets
train_dataset = GraphDataset(root_dir, source_dir, train_list, fake_perc=fake_perc,test=False)
val_dataset = GraphDataset(root_dir, source_dir, val_list, fake_perc=fake_perc,test=True)

## Defining the data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Calculating the min, max values of node, edge attributes and target for scaling purposes
xmax,ymax,emax,xmin,ymin,emin = get_max_min(train_dataset)

# GNN Model Architecture

Next section contains the functions and classes which define the Multi-fidelity Graph U-Net architecture

Classes EdgeModel and NodeModel contain the edge and node update MLP functions for message passing and propagation

Net_EAGNN class contains the network architecture for a GNN model. It has the similar architecture as that of Meshgraphnet



In [ ]:
class EdgeModel (torch.nn.Module):
    """
    Class for updating the edge attributes of the graph. 
    
    It is a MLP network with a single hidden layer, with ReLU activation function
    Input consists of edge attributes node attributes of the edges and the node attributes of the two nodes connected by the edge
    
    Output is the updated edge attribites of all the edges of the graph
    
    If residuals is True, the updated values are added to the previous edge attributes before they are returned.
    This is similar to the residual network.
    
    """
    
    def __init__(self, n_features, n_edge_features, hiddens, n_targets, residuals):
        super().__init__()
        self.residuals = residuals
        self. edge_mlp = Seq(
            Linear(2*n_features + n_edge_features, hiddens),
            ReLU(),
            Linear (hiddens, n_targets),
        )
        
    def forward(self, src, dest, edge_attr, u=None, batch=None):
        #Concats the nodes connecting the edges and edge attributes and passed through MLP
        #src and dest are the node attributes of the two nodes
        #edge_attr is the edge attributes of the edge connecting the two nodes
        
        out = torch.cat([src, dest, edge_attr], 1)
        out = self.edge_mlp(out)
        if self.residuals:
            out = out + edge_attr
        return out

class NodeModel (torch.nn.Module):
    """
    Class for updating the node attributes of the graph. 
    
    It consists of two MLP networks. Both the networks have single hidden layer with with ReLU activation function.
    
    First MLP network, node_mlp_1, takes the node attributes of the neighboring nodes and the updated edge attributes of the
    edges connecting these nodes as the input. Output is the message from the neighboring nodes, with the same size as the
    node attributes. 
    
    Second MLP network, node_mlp_2, takes the aggregated message acorss all the neighboring nodes from node_mlp_1 
    and the node attributes of the current node as the input. Output is the updated node attributes. 
    
    If residuals is True, the updated values are added to the previous node attributes before they are returned.
    This is similar to the residual network.
    """ 
    
    def __init__(self, n_features, n_edge_features, hiddens, n_targets, residuals):
        super(NodeModel, self).__init__()
        
        self.residuals = residuals
        
        #message calculation MLP
        self. node_mlp_1 = Seq(
            Linear(n_features + n_edge_features, hiddens),
            ReLU(),
            Linear(hiddens, n_targets),
        )
        
        #node attribute update MLP
        self.node_mlp_2 = Seq(
            Linear (hiddens + n_features, hiddens),
            ReLU(),
            Linear(hiddens, n_targets),
        )

    def forward(self, x, edge_index, edge_attr, u, batch):
        row, col = edge_index
        out = torch. cat([x[col], edge_attr], dim=1)
        out = self.node_mlp_1(out) #message calculation
        out = torch_scatter.scatter_add(out, row, dim=0, dim_size=x.size(0)) #message aggregation, aggregation function is SUM
        out = torch.cat([x, out], dim=1)
        out = self.node_mlp_2(out) #node update
        if self.residuals:
            out = out + x
        return out


def build_layer(n_features, n_edge_features, hiddens, n_targets, batchnorm=False, residuals=True):
    """Calling the edge and node update modules
    """ 
    return geom_nn.MetaLayer(
        edge_model=EdgeModel(n_features, n_edge_features, hiddens, n_targets, residuals=residuals),
        node_model=NodeModel(n_features, n_edge_features, hiddens, n_targets, residuals=residuals),
    )




In [ ]:
class Net_EAGNN(torch.nn.Module):
    """
    Class defining the GNN architecture
    
    """ 
    def __init__(self, node_att_num, edge_att_num, output_dim, 
                 gn_depth, encoder_neurons, node_index_list, 
                 dropout=0.1, batch_norm = False):
        super(Net_EAGNN, self).__init__()

        self.node_att_num = node_att_num #number of node attributes
        self.edge_att_num = edge_att_num #number of edge attributes
        self.encoder_neurons = encoder_neurons #number of neurons in each layer of encoder network
        self.output_dim = output_dim #number of outputs/targets
        self.gn_depth = gn_depth #list containing the number of GN blocks for each level
        self.node_index_list = node_index_list #indices of the node features to be used for training
        self.dropout = dropout #droput ratio
        self.channel = self.encoder_neurons[1] #latent space size after encoding
        # defining the encoder network for node attributes
        self.encoder_node = Seq(Linear(self.node_att_num, self.encoder_neurons[0]),
                                ReLU(),
                                Linear(self.encoder_neurons[0], self.encoder_neurons[1]))
        # defining the encoder network for edge attributes
        self.encoder_edge = Seq(Linear(self.edge_att_num, self.encoder_neurons[0]),
                                ReLU(),
                                Linear(self.encoder_neurons[0], self.encoder_neurons[1]))
        # defining the decoder network 
        self.decoder = Seq(Linear(self.encoder_neurons[1], self.encoder_neurons[0]),
                                ReLU(),
                                Linear(self.encoder_neurons[0], self.output_dim))
        # defining batch norm if used
        self.bn = torch.nn.BatchNorm1d(self.channel)
        
        # defining the GN blocks for message passing and aggregation
        self.GN_Blocks= torch.nn.ModuleList()
        for i in range(self.gn_depth):
            self.GN_Blocks.append(build_layer(self.channel,self.channel,self.channel,self.channel))
        
    def forward(self, data, device, scale=True):
                
        data = data.to(device)

        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        
        if scale:
            # scaling node attributes
            x = min_max_scaler(x, xmin, xmax, device)
            # scaling edge attributes
            edge_attr = min_max_scaler(edge_attr, emin, emax, device)

        # encoding the node attributes for the graphs
        x = self.encoder_node(x[:,self.node_index_list])
        
        # encoding the edge attributes for the graphs
        edge_attr = self.encoder_edge(torch.tensor(edge_attr, dtype=torch.float))
        
        #  node attributes of coarse graph are  passed through the entire set of GN blocks
        for i in range(self.gn_depth_list):
            x, edge_attr, _ = self.GN_Blocks[i](x, edge_index, edge_attr=edge_attr)
            
        x = self.decoder(x)
        
        return x

# Model training and evaluation

In [ ]:
def model_train(data_loader, loss_all, device, scale):
    """Training the GNN model
    
    Parameters
    -----------
    data_loader : Data loader object from pytorch geometric, it contains all the graphs for training
    loss_all : loss value, Tensor float
    device : GPU/CPU
    scale : if True, scaling is done on node and edge attributes as well as the target, boolean
    
    Ouput
    ------
    loss_all : loss value after a single epoch, Tensor float
       
    """ 
    model.train()
    for data in data_loader:
        # get the predicted outputs
        out = model(data, device, scale)
        
        optimizer.zero_grad(set_to_none=True)
        
        # scale target responses if scale is True
        if scale:
            y =  min_max_scaler(data.y, ymin, ymax, device).reshape(-1,1) #change the reshape based on the number of outputs
        else:
            y = data[0].y.reshape(-1,1)
            
        # loss calculation
        loss_calc = loss(out.reshape(-1,1), y.reshape(-1,1)) 
        loss_calc.backward()
        
        optimizer.step()
        my_lr_scheduler.step()
        
    return loss_all


def model_eval(data_loader, device, scale):
    """Evaluating the GNN model
    
    Parameters
    -----------
    data_loader : Data loader object from pytorch geometric, it contains all the graphs for training
    device : GPU/CPU
    scale : if True, scaling is done on node and edge attributes as well as the target, boolean
    
    Ouput
    ------
    l2_err : relative L2 error for the predictions in the graph, float
       
    """ 
    model.eval()
    
    predictions = []
    labels = []
    
    for data in data_loader:
                
        #getting the prediction from just the fine graph
        pred = model(data, device, scale)
        
        if scale:
            pred = min_max_descaler(pred, ymin, ymax, device).detach().cpu().numpy().reshape(-1,1)
        else:
            pred = pred.detach().cpu().numpy().reshape(-1,1)
            
        label = data.y.detach().cpu().numpy().reshape(-1,1)
        predictions.append(pred)
        labels.append(label)
        
    predictions = np.vstack(predictions)
    labels = np.vstack(labels)
    
    # calculation of relative L2 error
    diff_norm = np.linalg.norm(predictions - labels, ord=2)
    y_norm = np.linalg.norm(labels, ord=2)
    l2_err = np.mean(diff_norm / y_norm)

    return l2_err

### CHANGE TRAINING AND MODEL PARAMETERS HERE!

In [ ]:

# TRAINING HYPERPARAMETERS
n_epochs = 1000
batch_size = 2
lr = 0.001
weight_decay=1e-6

# MODEL PARAMETERS
# change these based on your data
node_att_num = 10
edge_att_num = 5
output_dim = 1
gn_depth = 12
encoder_neurons = [64, 128]
node_index_list = [0,1,4,5,6,7,8,9,10,11]
dropout = 0.0
scale = True
batch_norm = False

# DIRECTORIES TO STORE RESULTS
result_dir = 'results'
model_dir = 'models'
loss_dir = 'losses'


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Net_EAGNN(node_att_num=node_att_num, edge_att_num=edge_att_num, output_dim=output_dim,
                gn_depth=gn_depth, node_index_list=node_index_list,encoder_neurons=encoder_neurons, 
                dropout=dropout, batch_norm = False).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.99, 0.999), weight_decay=weight_decay)
my_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=n_epochs, eta_min=1e-8)


loss = torch.nn.MSELoss()


### MODEL TRAINING

In [ ]:
## model training
epoch_list = []
train_l2_err = []
val_l2_err = []

print('Training started...')

for epoch in range(n_epochs):
    loss_all = 0
    loss_all = model_train(train_loader, loss_all, device, scale)
    if(epoch%10==0):
        epoch_list.append(epoch)
        l2_err = model_eval(train_loader, device, scale)
        train_l2_err.append(l2_err)
        l2_err = model_eval(val_loader, device, scale)
        val_l2_err.append(l2_err)
        print('epoch: ', epoch, 'train error: ', train_l2_err[-1], 'val error: ', val_l2_err[-1])
        print()

        # saving the model
        torch.save({
            'epoch':epoch,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
        }, result_dir + '/' + model_dir + '/model_eagnn.pt')
        
        # saving the loss results
        np.savetxt(result_dir + '/' + loss_dir + '/train_l2_err.txt', train_l2_err)
        np.savetxt(result_dir + '/' + loss_dir + '/val_l2_err.txt', val_l2_err)
        